# Text-cleaning tasks

## Routines to go through text and correct misspellings / OCR errors

Updating text throughout the 'trimmed' corpus tree.

a global change will require scrubbing through (text-data/):

- CC_ML_FR_trimmed_morphad_lem
- CC_ML_FR_trimmed_morphad_lem_AMER
- CC_ML_FR_trimmed_MAlem_AMER_cat_mod
- CC_ML_FR_trimmed_MAlem_AMER_cat_mod_enc_full_part

An American change will require scrubbing through the American portion of the first two and then through all the rest.

Depending on testing, the unlemmatized **Cambridge_MacLehose_FineReader_OCR_trimmed** may be treated as part of global or separately

To identify words, priorities & uses, I will need textcollections and freqdists for raw, total and AMER slices

If I decide on a programmatic scrub (~for X top tokens, collapse everything within cloze fuzzy range into main token), I'll probably want to apply it without filtering stop words

In [13]:
import os
import re
import nltk
from nltk.corpus import PlaintextCorpusReader
from fuzzywuzzy import fuzz
from fuzzywuzzy import process 

root = 'text-data/'

am_root = root + 'CC_ML_FR_trimmed_morphad_lem_AMER/'
am_cat_root = root + 'CC_ML_FR_trimmed_MAlem_AMER_cat'
raw_root = root + 'Cambridge_MacLehose_FineReader_OCR_trimmed'
tot_root = root + 'CC_ML_FR_trimmed_morphad_lem'
trim_root = root + 'Cambridge_MacLehose_FineReader_OCR_trimmed/'

In [5]:
from nltk.corpus import stopwords
from nltk import ngrams

eliz_stopwords = ["i",  "me",  "my",  "myself",  "we",  "our",  "ours",  "ourselves",  "you",  "your",  "yours",  "yourself",  "yourselves",  "he",  "him",  "his",  "himself",  "she",  "her",  "hers",  "herself",  "it",  "its",  "itself",  "they",  "them",  "their",  "theirs",  "themselves",  "what",  "which",  "who",  "whom",  "this",  "that",  "these",  "those",  "am",  "is",  "are",  "was",  "were",  "be",  "been",  "being",  "have",  "has",  "had",  "having",  "do",  "does",  "did",  "doing",  "a",  "an",  "the",  "and",  "but",  "if",  "or",  "because",  "as",  "until",  "while",  "of",  "at",  "by",  "for",  "with",  "about",  "against",  "between",  "into",  "through",  "during",  "before",  "after",  "above",  "below",  "to",  "from",  "up",  "down",  "in",  "out",  "o",  "on",  "off",  "over",  "under",  "again",  "further",  "then",  "once",  "here",  "there",  "when",  "where",  "why",  "how",  "all",  "any",  "both",  "each",  "few",  "more",  "most",  "other",  "some",  "such",  "no",  "nor",  "not",  "only",  "own",  "same",  "so",  "than",  "too",  "very",  "can",  "will",  "just",  "should",  "now",  "art", "doth", "dost", "'ere", "hast", "hath", "hence", "hither", "nigh", "oft", "should'st", "thither", "thee", "thou", "thine", "thy", "'tis", "'twas", "wast", "whence", "wherefore", "whereto", "withal", "would'st", "ye", "yon", "yonder"]
hk_stopwords = ['unto','u','one', 'five','upon','de','also','wee','two','may','many','would','shall','hee','like','three','doe','could','much','every','againe','bee','might','without','well','within','yet','bene','ad','foure','another','whereof','thereof','onely','next','himselfe','thus','hundred','untill','therefore','halfe','cum','selfe','non','ut', 'whole','little','sixe','full','neither','among','last','c','never','la','qui','ii','according','eight','whose','either','per','along','item','al','likewise','mee','whereupon','none','till','able','thousand','self','el','second','que','mine','quae','sunt','et','seven','iii','although','litle','si','notwithstanding','besides','etiam','lesse','e','even','vel','alwayes', 'third','ever','rather','whether','still','otherwise','large','amongst', 'greater','somewhat','ex','least','aforesaid','though','whatsoever','quam', 'ten','whereby','foorth','no', 'n','los','almost','twelve','howbeit','j', 'greatly','ac','yce', 'pro','en','ab','greatest','whereas','hoc','w','beene','doeth','eorum','con','withall','hereafter','moreover','nec','nine','noone','omnes','del','enim','often']
latin_stopwords = ['ab', 'ac', 'ad', 'adhuc', 'aliqui', 'aliquis', 'an', 'ante', 'apud', 'at', 'atque', 'aut', 'autem', 'cum', 'cur', 'de', 'deinde', 'dum', 'ego', 'enim', 'ergo', 'es', 'est', 'et', 'etiam', 'etsi', 'ex', 'fio', 'haud', 'hic', 'iam', 'idem', 'igitur', 'ille', 'in', 'infra', 'inter', 'interim', 'ipse', 'is', 'ita', 'magis', 'modo', 'mox', 'nam', 'ne', 'nec', 'necque', 'neque', 'nisi', 'non', 'nos', 'o', 'ob', 'per', 'possum', 'post', 'pro', 'quae', 'quam', 'quare', 'qui', 'quia', 'quicumque', 'quidem', 'quilibet', 'quis', 'quisnam', 'quisquam', 'quisque', 'quisquis', 'quo', 'quoniam', 'sed', 'si', 'sic', 'sive', 'sub', 'sui', 'sum', 'super', 'suus', 'tam', 'tamen', 'trans', 'tu', 'tum', 'ubi', 'uel', 'uero']
pronouns = ['i', 'you', 'he', 'she', 'it', 'we', 'you', 'they', 'me', 'you', 'him', 'her', 'it', 'you', 'us', 'them', 'my', 'your', 'his', 'her', 'its', 'our', 'your', 'their', 'mine', 'yours', 'his', 'hers', 'its', 'ours', 'yours', 'theirs', 'myself', 'yourself', 'himself', 'herself', 'itself', 'ourselves', 'yourselves', 'themselves']
modals = ['shall','shal','shalt','should', 'can', "can't", 'cannot', 'could', 'will','wil', 'would', 'may', 'must', 'might', 'ought', 'need', 'have', 'has']
directives = ['without', 'within', 'there', 'thence','away','est','towards','toward','farre','betweene','wherein','therein']
stop_words = set(stopwords.words('english') + latin_stopwords + eliz_stopwords + hk_stopwords + ['']) - set(pronouns + modals + directives)
stop_words_max = set(stopwords.words('english') + latin_stopwords + eliz_stopwords + hk_stopwords + [''] + pronouns + modals + directives + ['great', 'make', 'good', 'part', 'certain', 'thing', 'wherewith', 'afterward', 'day'])



In [5]:
# #testing
# s = 'whose care hath bene more generali 5people. And how may it be thought ;5people, generali? generaliii'
# old_test = '5people'
# new_test = 'NEW'
# print(re.sub(r'([^a-zA-Z]|^)('+ old_test + r')([^a-zA-Z])',r'\1' + new_test +r'\3', s, flags=re.I))

whose care hath bene more generali NEW. And how may it be thought ;NEW, generali? generaliii


In [9]:
def replace(old, new, scope):
    '''
    replaces all instances of 'old' token with 'new' token across corpuses defined through 'scope'
    arguments:
        old (str) : token to be replaced
        new (str) : substitute token
        scope (str) : 
            'am' : American materials
            'tot' : all lemmatized materials
            'raw' : unlemmatized (complete collection)
            'totnraw' : both lemmatized and unlemmatized folders '''
    #since all but 'raw' scope includes the regrettably fragmented American materials, pre-load them 
    folders = [am_root, am_cat_root]
    if scope == 'raw':
        folders = [raw_root]
    elif scope == 'am':
        folders += []
    elif scope == 'tot':
        folders += [tot_root]
    elif scope == 'totnraw':
        folders += [tot_root, raw_root]
    elif scope == 'test':
        folders = [root + 'test/']
    else:
        raise ValueError('unrecognized scope argument')
    #print(folders)
    for folder in folders:
            filelist = os.scandir(folder)
            for entry in filelist:
                with open(entry.path, 'r', encoding="utf8") as fr:
                        text = fr.read()
                        # replacement through regex to account for adjacent punctuation & ignore case
                        # regex fails if 'new' starts with a number; prefacing the number with a space gets around that
                        text = re.sub(r'([^a-zA-Z]|^)('+ old + r')([^a-zA-Z])',r'\1' + new +r'\3', text, flags=re.I)
                with open(entry.path, 'w', encoding="utf8") as fw:
                        fw.write(text)
    with open (root + 'replacement_record.txt', 'a', encoding="utf8") as f:
        f.write(old + ' -> ' + new + ' in ' + scope + '\n')

In [74]:
#extra layer of cleaning, including placeholder newlines for morphadorner
filelist = os.scandir('text-data/CC_ML_FR_trimmed_cleaned_EN/')
for entry in filelist:
    with open(entry.path, 'r', encoding="utf8") as fr:
        text = fr.read()
        text = text.replace('THE ENGLISH VOYAGES', '')
        text = re.sub(r'\[.{1,9}?\]', '', text)
        text = re.sub(r'\n+', r'{n}', text)
        with open('text-data/CC_ML_FR_trimmed_cleaned_EN_cl/' + entry.name, 'w', encoding="utf8") as fw:
            fw.write(text)

In [76]:
#cleaning out the { n } in MA outputs
folder = 'text-data/CC_ML_FR_trimmed_cleaned_EN_cl_MAlem'
filelist = os.scandir(folder)
for entry in filelist:
    with open(entry.path, 'r+', encoding="utf8") as f:
        text = f.read()
        text = text.replace('{ n }', '\n')
        f.seek(0)
        f.write(text)
        f.truncate()

### Exploring targets for spelling standardization 

In [6]:
#corpuses
am = PlaintextCorpusReader(am_root, '.*')
am_fd = nltk.FreqDist(word for word in am.words() if word.isalpha() and word not in stop_words)
am_col = nltk.TextCollection(am)

raw = PlaintextCorpusReader(raw_root, '.*')
raw_fd = nltk.FreqDist(word for word in raw.words() if word.isalpha() and word not in stop_words)
raw_col = nltk.TextCollection(raw)

tot = PlaintextCorpusReader(tot_root, '.*')
tot_fd = nltk.FreqDist(word for word in tot.words() if word.isalpha() and word not in stop_words)
tot_col = nltk.TextCollection(tot)

In [45]:
# # testing
# with open('text-data/test/11_30_AMER_1587_The_prosperous_voyage_of_M_Thomas_Candish_esquire_into_the_South_sea_and_so_round_about_the_circumfe_pp.290-347.txt', 'r', encoding="utf8") as fr:
#     text = fr.read()
#     text = text.replace('THE ENGLISH VOYAGES', '')
#     text = re.sub(r'\[.{1,9}?\]', '', text)
#     text = re.sub(r'\n+', r'{n}', text)
#     print(text)

# test = "The state of the shipping of the Cinque ports from Edward the Confessor and William the Conqueror , and so down to Edward the first , faithfully gathered by the learned Gentleman 42 { n } M. Willaim Lambert in his Perambulation of Kent , out of the most ancient Records of England . { n } Find in the book of the general ! survey The antiquity of the Realm , which William the Con - °fthe Porls-querour caused to be made in the fourth IO7°-yeere of his reign , and to be called Domesday , because ( as Matthew Parise says ) it spared no man but judged all men indifferently , as the Lord in that great { n } day will do , that Dover , Sandwich , and Rumney , were in the time of K. Edward the Confessor , discharged almost of all manner of impositions and burdens ( which other towns did bear ) in consideration of such service to be done by them upon the Sea , as in their special titles shall hereafter appear . { n } Whereupon , although I might ground reasonable conjecture , that the immunity of the haven Towns ( which we now call by a certain number , the Cinque Ports ) might take their beginning from the same Edward : yet for as much as I read in the Chartre of K. Edward the first after the conquest ( which is reported in our book of Entries ) A reci tall of the grants of sundry kings to the Five Ports , the same reaching no higher then to William the Conqueror , I will leave my conjecture , and lean to his Chartre : contenting my self to yield to the Conqueror , the thanks of other men's benefits , seeing those which were benefited , were wisely contented ( as the case then stood ) to like better of his confirmation ( or second gift ) then of K. Edwards first grant , and endowment . { n } And to the end that I may proceed in some manner of array , I will first show , which Towns were at the beginning taken for the Five Ports , and what others be now reputed in the same number : secondly , what service they ought , and did in times passed : and lastly , what privileges they have therefore , and by what persons they have be governed ."
# print(test.replace('{ n }', '\n'))

In [38]:
viol_flags = ['violence', 'harm', 'injury', 'injure', 'hurt', 'damage', 'scathe', 'wound', 'maim', 'cripple', 'mutilate', 'cut', 'mangle', 'torture', 'torment', 'wound', 'gash', 'bruise', 'abuse', 
              'bloody', 'bloodshed', 'bloodshedder', 'bloodshedding', 'blood', 'hit', 
              'fight', 'scrap', 'struggle', 'conflict', 'melee', 'brawl', 'combat', 'wrestle', 'wrestler',
              'kill', 'death', 'slay', 'murder', 'assassinate', 'assasin', 'massacre', 'slaughter', 'butcher', 'slaughter', 'manslaughter', 
              'battle', 'war', 'siege', 'attack', 'assault', 'skirmish', 'skirmisher', 'enemy', 'foe', 'hostile', 'army', 'soldier', 'warrior', 'conquer', 'conqueror',  'conquest'
              'detain', 'capture', 'captive', 'imprison', 'gaol', 'prisoner', 'slave', 'enslave',
              'shoot', 'shot',  'blast', 'burn', 'fire', 'blaze',
              'cruel', 'cruelty', 'destroy', 
              'arrow', 'crossbow', 'dart', 'javelin', 'mace', 'club', 'sword', 'lance', 'spear', 'rapier', 'pike', 'target', 'buckler', 'falchion', 'halberd', 'partisan', 
              'musket', 'gun', 'bullet', 'caliver', 'culverin', 'harquebus', 'harquebusier', 'saker', 'cannon']

In [12]:
tot_fd.most_common(50)

In [66]:
tot_col.concordance('pemisapans', 200)

Displaying 4 of 4 matches:
 like to be starve , but the other false . nevertheless until my return it take such effect in pemisapans breast , and in those against we , that they grow not only into contempt of we , but also ( co
e attempt to run away , i lay he in the bylboe , threaten to cut off his head , who i remit at pemisapans request : whereupon he be persuade that he be our enemy to the death , he do not only feed he 
ake much of he , he flat discover all unto i , which also afterward be reveal unto i by one of pemisapans own man , that night before he be slay . Theise mischief be all instant upon i and my company 
gligence to have be intercept by the savage , we meet he Pemisapan return out of the wood with pemisapans head in slay - he hand .. This fall out the first of June 1586 , and the eight of the same com


In [7]:
# 86 threshold for 'people' 'harquebus', 'harquebuss',
for word in tot_col.vocab():
    if fuzz.ratio('harquebus', word) > 70:
        print(word)

harquebus
rebus
harquebusier
haeredibus
Marques
marquess
arcubus
haquinus
harque
hargubuser
harebornus
hargubuz
harquebusiers
Marquese
harquebusses


In [14]:
replace('harquebuss','harquebus','totnraw')

## clearing out Latin text

### false starts

In [29]:
# # First attempt - manual flagging
# # with open(text-data/comaprison_texts/latin.txt, 'r+', encoding="utf8") as f:
# #     latin = f.read()
    
# latin = PlaintextCorpusReader('text-data/comparison_texts', 'latin.txt')
# latin_fd = nltk.FreqDist(word for word in latin.words() if word.isalpha())
# latin_fd.most_common(60) 

# latin_flags = ['ad', 'de', 'est', 'quod', 'cum', 'non', 'ut', 'qui', 'sunt', 'et', 'Et', 'etiam', 'quae', 'vel', 'autem', 'quam', 'eorum', 'sed', 'quia', 'eos', 'nos', 'enim', 'hoc', 'se', 'nec', 'ab', 'omnes', 'nobis', 'homines', 'vero', 'tamen', 'eis', 'esse', 'pro', 'habent', 'usque', 'terram', 'sicut', 'terra', 'De', 'illa', 'nisi', 'faciunt', 'quo', 'unum', 'quibus', 'habet', 'ita', 'ejus', 'ibi']
# print(len(latin_flags))

# filelist = os.scandir('text-data/CC_ML_FR_trimmed_cleaned')
# lat_texts = 0
# for entry in filelist:
#     with open(entry.path, 'r', encoding="utf8") as fr:
#         text = fr.read()
#         lat_count = 0
#         for flag in latin_flags:
#             if flag in text:
#                 lat_count += 1
#         if lat_count >= 15:
#             print(entry.name)
#             lat_texts += 1
            
# print(lat_texts)

50


In [37]:
# # check 'the same in English' expression flag
# filelist = os.scandir('text-data/CC_ML_FR_trimmed_cleaned')
# sameng = 0
# for entry in filelist:
#     with open(entry.path, 'r', encoding="utf8") as fr:
#         text = fr.read()
#         if 'same in English' in text:
#             sameng+=1
# print(sameng)

88


In [40]:
# #second attempt through TextBlob's language detection
# from textblob import TextBlob

In [41]:
# with open('text-data/Cambridge_MacLehose_FineReader_OCR_trimmed/01_01_NNE-_0517_The_voyage_of_Arthur_K_of_Britaine_to_Island_and_the_most_Northeastern_parts_of_Europe_Anno_517_pp.003-004.txt', 'r', encoding="utf8") as fr:
#     text = fr.read()
# sents = text.split('\n')
# for sent in sents:
#     lang = TextBlob(sent)
#     print(lang)
#     print(lang.detect_language())

In [44]:
# lang = TextBlob('bob')
# print(lang)
# print(lang.detect_language())

### langid implementation

In [54]:
import langid
from langid.langid import LanguageIdentifier, model
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
identifier.set_languages(['en','la','es'])

In [70]:
# # #test runs
# with open('text-data/CC_ML_FR_trimmed_cleaned/11_38_AMER_1596_The_letters_of_the_Queenes_most_excellent_Majestie_sent_in_the_yere_1596_to_the_emperour_of_China_by_pp.417-421.txt', 'r', encoding="utf8") as fr:
#     text = fr.read()
# sents = text.split('{n}')
# for sent in sents:
#     if len(sent) > 2:
#         print(sent)
#         print(langid.classify(sent))

In [73]:
# dumps non-EN text
source = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat'
target = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat_EN/'
filelist = os.scandir(source)
for entry in filelist:
    with open(entry.path, 'r', encoding="utf8") as f:
        text = f.read()
        sents = text.split('{n}')
        EN_sents = []
        for sent in sents:
            if langid.classify(sent)[0] == 'en':
                EN_sents.append(sent)
        EN_text = '{n}'.join(EN_sents)
        # control for small bits of English misidentified
        if len(EN_text) > 0.7* len(text):
            EN_text = text
        with open(target + entry.name, 'w', encoding="utf8") as fw:
            fw.write(EN_text)

**having created a base EN corpus, I will want to create the following:**
- ~EN version of CC_ML_FR_trimmed_MAlem_AMER_cat~
- ~MorphAdorn the base EN~
  - ~from that, create lem and spel versions; convert {n} to newlines~
- recreate ledger with just EN wordcounts; run viol ratios
- create AM lem folder
- MALLET general lem / AM lem at higher topic numbers in case it gets me a fighting topic